# **Scraping Coretax News Content**

Web scraping di sini menggunakan `BeautifulSoup` karena library ini memudahkan parsing dan navigasi HTML, serta ekstraksi data seperti teks dan judul artikel. `BeautifulSoup` digunakan bersama `requests` untuk mengambil konten web.

## 🔧 Libraries Used

In [10]:
# Import Library
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime

## 📥 Import Data

In [11]:
# Import data
file_path = "../data/Kelompok 2 Link Article Coretax.xlsx"
ctx_df = pd.read_excel(file_path)
ctx_df

,NO,SUMBER,PENERBIT,LINK,JUDUL
0,1,local news,Suara Merdeka,https://diorama.suaramerdeka.com/tech/18315058...,"Coretax Error Hari Ini, 30 April 2025 Cara Per..."
1,2,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,Dirjen Pajak Ungkap Kabar Terbaru soal Perbaik...
2,3,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,DJP Klaim Aplikasi Coretax Mulai Stabil Digunakan
3,4,local news,Kumparan,https://kumparan.com/kumparanbisnis/dirjen-paj...,Dirjen Pajak Klaim Perbaikan Sistem Coretax Su...
4,5,local news,DDTC,https://news.ddtc.co.id/berita/nasional/181042...,"Latensi Turun, DJP Klaim Coretax System Sudah ..."
...,...,...,...,...,...
156,157,economic,Tempo,https://www.tempo.co/ekonomi/luhut-sebut-siste...,"Luhut Sebut Sistem Coretax akan Membaik, Apa i..."
157,158,economic,MSN,https://www.msn.com/id-id/ekonomi/ekonomi/core...,"Coretax Kacau, tapi Kenapa Tak Dibahas dalam A..."
158,159,economic,Oke Zone,https://economy.okezone.com/read/2025/04/24/32...,Sri Mulyani Pamer Setoran Pajak Naik Jadi Rp13...
159,160,economic,Bisnis,https://ekonomi.bisnis.com/read/20250423/259/1...,Ditjen Pajak Klaim Performa Coretax Sudah Stabil


Output yang ditampilkan adalah dataset yang berisi artikel-artikel mengenai **CoreTax** dari berbagai sumber berita. Terdapat 5 kolom pada dataset, yaitu:
1. **NO**: Nomor urut artikel.
2. **SUMBER**: Kategori sumber berita 
3. **PENERBIT**: Nama penerbit atau organisasi yang mempublikasikan artikel.
4. **LINK**: URL untuk mengakses artikel tersebut.
5. **JUDUL**: Judul artikel yang membahas mengenai CoreTax.

Dataset ini berisi 150 baris data yang terdiri dari berbagai artikel mengenai **CoreTax**.

# Web Scraping

In [12]:
# Fungsi untuk menghapus teks iklan, "baca juga", dll.
def clean_content(content):
    # Hapus semua teks yang mengandung "baca juga", "iklan", atau yang sejenis
    content = re.sub(r'\b(Baca Juga|Iklan|ADVERTISEMENT|baca juga|IKLAN)\b', '', content, flags=re.IGNORECASE)
    return ' '.join(content.split())  # Hapus spasi berlebih, jadi satu baris

In [13]:
def get_article_content(link, retries=3):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
        }
        response = requests.get(link, headers=headers, timeout=30)

        if response.status_code != 200:
            return None, None, None, "failed"

        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')
        content = ' '.join([p.get_text() for p in paragraphs]).strip()

        # Ambil tag
        meta_tags = soup.find('meta', attrs={'name': 'keywords'})
        tags = meta_tags['content'] if meta_tags and 'content' in meta_tags.attrs else "Tidak ada tags"

        # === Ambil tanggal ===
        date = None

        # Coba beberapa cara
        time_tag = soup.find('time')
        if time_tag and time_tag.get('datetime'):
            date = time_tag['datetime']
        elif time_tag:
            date = time_tag.get_text()
        else:
            # Coba cari berdasarkan class umum
            date_tag = soup.find(['span', 'p', 'div'], class_=['date', 'published', 'meta-date', 'entry-date'])
            if date_tag:
                date = date_tag.get_text().strip()

        if len(content) == 0:
            return None, None, None, "failed"

        return content, tags, date, "success"

    except requests.exceptions.RequestException as e:
        if retries > 0:
            print(f"Retrying due to error: {e}. {retries} attempts left.")
            time.sleep(2)
            return get_article_content(link, retries-1)
        return None, None, None, f"failed after retries: {e}"

    except Exception as e:
        print(f"Error lainnya pada {link}: {e}")
        return None, None, None, "failed"


In [14]:
# Menambah kolom baru untuk konten, tags, date, dan status
ctx_df['content'] = None
ctx_df['tags'] = None
ctx_df['date'] = None
ctx_df['status'] = None

In [15]:
# Loop untuk mengambil isi berita dari setiap link
success_count = 0
failed_count = 0

for index, row in ctx_df.iterrows():
    link = row['LINK']

    try:
        # Ambil semua 4 nilai dari fungsi get_article_content
        content, tags, date, status = get_article_content(link)

        # Menyimpan hasil dalam DataFrame
        ctx_df.at[index, 'content'] = content if content else "Tidak ada konten"
        ctx_df.at[index, 'tags'] = tags if tags else "Tidak ada tags"
        ctx_df.at[index, 'date'] = date if date else "Tidak ada tanggal"
        ctx_df.at[index, 'status'] = status

        # Menghitung keberhasilan dan kegagalan
        if status == "success":
            success_count += 1
        else:
            failed_count += 1

    except requests.exceptions.RequestException as e:
        print(f"Error mengambil konten dari {link}: {e}")
        ctx_df.at[index, 'content'] = "Error mengambil konten"
        ctx_df.at[index, 'tags'] = "Error mengambil tags"
        ctx_df.at[index, 'date'] = "Error tanggal"
        ctx_df.at[index, 'status'] = "failed"
        failed_count += 1

    except Exception as e:
        print(f"Error lainnya pada {link}: {e}")
        ctx_df.at[index, 'content'] = "Error lainnya"
        ctx_df.at[index, 'tags'] = "Error lainnya"
        ctx_df.at[index, 'date'] = "Error tanggal"
        ctx_df.at[index, 'status'] = "failed"
        failed_count += 1

print(f"Jumlah artikel yang berhasil diambil: {success_count}")
print(f"Jumlah artikel yang gagal diambil: {failed_count}")


Jumlah artikel yang berhasil diambil: 153
Jumlah artikel yang gagal diambil: 8


## 📌 Save Data

In [17]:
# Simpan DataFrame
output_path = "../data/hasil_scrap_berita.xlsx"
ctx_df.to_excel(output_path, index=False)

In [18]:
ctx_df

,NO,SUMBER,PENERBIT,LINK,JUDUL,content,tags,date,status
0,1,local news,Suara Merdeka,https://diorama.suaramerdeka.com/tech/18315058...,"Coretax Error Hari Ini, 30 April 2025 Cara Per...",Tidak ada konten,Tidak ada tags,Tidak ada tanggal,failed
1,2,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,Dirjen Pajak Ungkap Kabar Terbaru soal Perbaik...,Direktur Jenderal (Dirjen) Pajak Suryo Utomo m...,"coretax,dirjen pajak,administrasi perpajakan,f...",Tidak ada tanggal,success
2,3,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,DJP Klaim Aplikasi Coretax Mulai Stabil Digunakan,Direktorat Jenderal Pajak (DJP) Kementerian Ke...,"coretax,djp,aplikasi perpajakan,pembaruan,ppnb...",Tidak ada tanggal,success
3,4,local news,Kumparan,https://kumparan.com/kumparanbisnis/dirjen-paj...,Dirjen Pajak Klaim Perbaikan Sistem Coretax Su...,Tidak ada konten,Tidak ada tags,Tidak ada tanggal,failed
4,5,local news,DDTC,https://news.ddtc.co.id/berita/nasional/181042...,"Latensi Turun, DJP Klaim Coretax System Sudah ...","Ilustrasi. JAKARTA, DDTCNews - Ditjen Pajak (D...","administrasi pajak, coretax, coretax system, DJP",Tidak ada tanggal,success
...,...,...,...,...,...,...,...,...,...
156,157,economic,Tempo,https://www.tempo.co/ekonomi/luhut-sebut-siste...,"Luhut Sebut Sistem Coretax akan Membaik, Apa i...",Scroll ke bawah untuk membaca berita Baca be...,Tidak ada tags,Tidak ada tanggal,success
157,158,economic,MSN,https://www.msn.com/id-id/ekonomi/ekonomi/core...,"Coretax Kacau, tapi Kenapa Tak Dibahas dalam A...",Tidak ada konten,Tidak ada tags,Tidak ada tanggal,failed
158,159,economic,Oke Zone,https://economy.okezone.com/read/2025/04/24/32...,Sri Mulyani Pamer Setoran Pajak Naik Jadi Rp13...,JAKARTA - Menteri Keuangan Sri Mulyani Indrawa...,Tidak ada tags,Tidak ada tanggal,success
159,160,economic,Bisnis,https://ekonomi.bisnis.com/read/20250423/259/1...,Ditjen Pajak Klaim Performa Coretax Sudah Stabil,Nyaman tanpa iklan. Langganan BisnisPro Nyaman...,"ditjen pajak, coretax, sistem pajak, sistem co...",Tidak ada tanggal,success


In [ ]:
# Menghapus baris yang memiliki status "failed"
ctx_df_cleaned = ctx_df[ctx_df['status'] != 'failed']

In [ ]:
ctx_df_cleaned

,SUMBER,PENERBIT,LINK,JUDUL,content,tags,status
0,local news,Tempo,https://www.tempo.co/ekonomi/penerimaan-pajak-...,"Penerimaan Pajak Merosot, APINDO: Hingga Kini ...","Baca berita dengan sedikit , klik di sini Baca...",Tidak ada tags,success
1,academic,UGM,https://ugm.ac.id/id/berita/coretax-masih-berm...,"Acap Dikeluhkan Wajib Pajak, Pengamat Ini Ungk...",Direktorat Jenderal Pajak (DJP) Kementerian Ke...,Tidak ada tags,success
2,local news,Hukum Online,https://www.hukumonline.com/berita/a/problemat...,Problematika CoreTax: Sistem Baru Pelaporan Pa...,Platform belajar hukum terbaik secara online d...,"ditjen-pajak,perpajakan,pajak,wajib-pajak",success
4,academic,Untirta,https://untirta.ac.id/2025/02/27/optimalisasi-...,Optimalisasi CoreTax di Lingkungan Akademik: L...,SERANG-Acara Optimalisasi Coretax di lingkunga...,Tidak ada tags,success
8,local news,Hukum Online,https://www.hukumonline.com/berita/a/gangguan-...,Gangguan pada Sistem AHU dan Masalah CoreTax H...,Platform belajar hukum terbaik secara online d...,"ahu,ahu-online,notaris,administrasi,fidusia,di...",success
...,...,...,...,...,...,...,...
143,opinion,Arah Kata,https://arahkata.pikiran-rakyat.com/hukrim/pr-...,Dugaan Korupsi Aplikasi CoreTax Rp13 Triliun K...,"ARAHKATA - Baru-baru ini, akses menuju sistem ...","KPK, korupsi, hukum, dugaan, Kemenkeu, dilapor...",success
144,local news,CNBC,https://www.cnbcindonesia.com/news/20250204160...,"Heboh Sistem CoreTax Jebol, Ditjen Pajak Buka ...","Jakarta, CNBC Indonesia - Pengguna Threads den...","pajak, coretax, npwp",success
145,opinion,NTVNews,https://www.ntvnews.id/ekonomi/0134120/pembuat...,Pembuatan Aplikasi CoreTax Senilai Rp 13 Trili...,"Ntvnews.id, Jakarta - Pembuatan aplikasi siste...","Coretax DJP,Coretax,pajak,kemenkeu,kpk",success
147,academic,UGM,https://ugm.ac.id/en/news/coretax-faces-challe...,"CoreTax Faces Challenges, UGM Economist Highli...","In early January, the Directorate General of T...",Tidak ada tags,success


In [ ]:
# Simpan DataFrame yang sudah dibersihkan ke dalam CSV dan Excel
output_csv = 'coretax_content_cleaned.csv'
output_excel = 'coretax_content_cleaned.xlsx'

ctx_df_cleaned.to_csv(output_csv, index=False)
ctx_df_cleaned.to_excel(output_excel, index=False)